In [1]:
from openpyxl import load_workbook
import sys
from StringIO import StringIO
import pandas as pd
import re


In [56]:
IGM_SAMPLE_METADATA_MAPPER = {
    'quantification':'Quantification Method',
    'library_prep':'Library Prep Method',
    'uid':'Sample Code',
    'size':'Library Size (bp)',
    'volume':'Volume (ml)',
    'concentration':'Conc. (nM)',
    'total_ng':'Total ng',
    'comment':'Comment',
    'index_1':'Index 1 (Name)',
    'index_2':'Index 2 (Name)',
}
IGM_EXPT_MAPPER = {
    'investigator':'Contact Name',
    'contact_email':'Contact Email',
    'study_date':'Date of Sample Submission',
    'institute':'Institute/Company Name',
    'pi_name':'PI Name',
    'pi_email':'PI Email',
    'member_of_MCC':'Member of MCC',
    'member_of_DRC':'Member of DRC',
    'po_number':'Index number or PO',
    'type_of_run':'Type of Run',
    'sequencing_platform':'Sequencing Platform',
    'number_of_cycles':'Number of cycles',
    'run_bioanalyzer_or_tapestation':'Run Bioanalyzer/Tape Station',
    'perform_qpcr':'Perform qPCR',
    'phix_spike_in_requested':'PhiX Spike In Requested?',
    'percentage_of_phix_if_applicable':'Percentage of PhiX if applicable',
    'indexing_system':'Indexing System',
    'total_number_of_lanes':'Total number of lanes',
    'total_number_of_samples':'Total number of samples'
}

In [11]:
def get_max_lines(sheet):
    """
    returns the max dimension (either row or col) of a sheet.
    """
    dim = sheet.calculate_dimension()
    return max(
        [int(i) for i in re.findall('[\d]+',dim)]
    )


def fill_sample_details_from_string(st, sheet, igm_sample_metadata_mapper, sep='\t'):
    df = pd.read_csv(st, sep=sep, index_col=0)
    offset = 1
    fields = df.columns
    for _, sample_id in df.iterrows():
        offset += 1
        for field in fields:
            row, col = get_tofill(sheet, field, offset=offset, horizontal=False)
            sheet.cell(row=row, column=col).value = sample_id[field]
        

In [12]:
def get_tofill(sheet, label, row_start=1, col_start=1, horizontal=True, offset=1):
    """
    returns the cell that needs to be filled given a field label.
    params:

    sheet: openpyxl.Worksheet
    label: Worksheet cell value
    row_start: int
        topmost value in workbook (1 = 1)
    col_start: int
        leftmost value in workbook (A = 1)
    horizontal:
        if True, return position of label + offset (to the right, horizontally)
        if False, return position of label + offset (downward, vertically)
    offset: int
    """
    for i in range(row_start, get_max_lines(sheet) + 1):
        for j in range(col_start, get_max_lines(sheet) + 1):
            field = sheet.cell(row=i, column=j).value
            if label == field:
                if horizontal:
                    return i, (j + offset)
                else:
                    return (i + offset), j
    print("found nothing for label: {}".format(label))
    return -1, -1

In [42]:
st=StringIO("""
Sample Name\tSample Code\tLibrary Size (bp)\tConc. (nM)\tVolume (ml)\tIndex 1 (Sequence)\n
Epool_e7.5\tEpool\t300\t6.46\t34\tA12\n
Epool_e7.6\tEpool2\t300\t7.46\t44\tA22\n
""")
active_sheet = 'Sample Information'
xlsx_template = '/home/bay001/projects/codebase/yaml2seq-manifest/data/template.xlsx'
wb = load_workbook(
    filename=xlsx_template
)
sheet = wb[active_sheet]

In [14]:
fill_sample_details_from_string(st, sheet, IGM_SAMPLE_METADATA_MAPPER)

In [15]:
wb.save('/home/bay001/projects/codebase/yaml2seq-manifest/data/output.xlsx')

In [10]:
st=StringIO("""
Sample Name\tSample Code\tLibrary Size (bp)\tConc. (nM)\tVolume (ml)\tIndex 1 (Sequence)\n
Epool_e7.5\tEpool\t300\t6.46\t34\tA12\n
Epool_e7.6\tEpool2\t300\t7.46\t44\tA22\n
""")
df = pd.read_csv(st, sep='\t', index_col=0)
df

,Sample Code,Library Size (bp),Conc. (nM),Volume (ml),Index 1 (Sequence)
Sample Name,,,,,
Epool_e7.5,Epool,300,6.46,34,A12
Epool_e7.6,Epool2,300,7.46,44,A22


In [59]:
st="""
investigator\tiachaim\n
contact_email\tiachaim@ucsd.edu\n
institute\tUCSD\n
study_date\t8/10/2017\n
"""
active_sheet = 'Sample Information'
xlsx_template = '/home/bay001/projects/codebase/yaml2seq-manifest/data/template.xlsx'
wb = load_workbook(
    filename=xlsx_template
)
sheet = wb[active_sheet]

In [72]:
def fill_expt_details_from_string(st, sheet, igm_expt_mapper, sep='\t'):
    df = pd.read_csv(StringIO(st), sep=sep, index_col=0, names=['value'])
    df.index = [igm_expt_mapper[c] for c in df.index]
    for field, _ in df.iterrows():
        print("field: ", field)
        row, col = get_tofill(
            sheet, field, offset=1, horizontal=True
        )
        print('add {} to: {}, {}'.format(df.ix[field][0], row, col))
    return df

In [73]:
fill_expt_details_from_string(st, sheet, IGM_EXPT_MAPPER)

('field: ', 'Contact Name')
add iachaim to: 8, 2
('field: ', 'Contact Email')
add iachaim@ucsd.edu to: 9, 2
('field: ', 'Institute/Company Name')
add UCSD to: 3, 2
('field: ', 'Date of Sample Submission')
add 8/10/2017 to: 2, 2


,value
Contact Name,iachaim
Contact Email,iachaim@ucsd.edu
Institute/Company Name,UCSD
Date of Sample Submission,8/10/2017
